Here we use Clifford simulations to confirm the formula for entanglement entropy growth.

In src/CliffordCircuits.jl we have useful functions for clifford simulations, defining Pauli operators usefully (but this is not relevant for understanding this notebook).

In [ ]:
include("src/CliffordCircuits.jl")
# using QuantumClifford
include("src/TreeStructure.jl")

First, we define useful 3-site gates and the corresponding layers of unitaries. Here we use three examples of Clifford tree-unitary gates with z=3: kicked Ising model, the Hadamard construction, and the triple SWAP (ie permutation of all 3 sites).

These operators take in 'triplets' as an argument, which is the sets of 3 nodes to which we apply the 3 site gates.

In [ ]:
function U_Ising!(ms::MixedDestabilizer,i::Integer,j::Integer)
    # Ising Hadamard term applied to two sites i, j
    # apply!(ms,random_clifford(2),[i,j])
    
    apply!(ms,tCNOT,[i,j])
    apply!(ms,tHadamard,[i])
    apply!(ms,tPhase,[j])
    apply!(ms,tHadamard,[i])
    apply!(ms,tCNOT,[i,j])
    
end

function U_kim!(ms::MixedDestabilizer,i::Integer,j::Integer,k::Integer)

    U_Ising!(ms,i,j) # ising term
    U_Ising!(ms,j,k) # ising term 
    apply!(ms,tHadamard,[i]) #kick on i
    apply!(ms,tHadamard,[j]) #kick on j
    apply!(ms,tHadamard,[k]) #kick on k
    U_Ising!(ms,i,j) # ising term
    U_Ising!(ms,j,k) # ising term 

end

function layer_U_kim!(ms, triplets::Vector{Vector{Int64}})
    # i=1
    for trip in triplets
        # println(i,trip)
        # i+=1
        U_kim!(ms,trip[1],trip[2],trip[3])
    end
end

function U_hadamard1!(ms::MixedDestabilizer,i::Integer,j::Integer,k::Integer)

    U_Ising!(ms,i,j) # ising term
    U_Ising!(ms,j,k) # ising term 
    U_Ising!(ms,i,k) # ising term 
    apply!(ms,tHadamard,[i]) #kick on i
    apply!(ms,tHadamard,[j]) #kick on j
    apply!(ms,tHadamard,[k]) #kick on k
    U_Ising!(ms,i,j) # ising term
    U_Ising!(ms,j,k) # ising term 
    U_Ising!(ms,i,k) # ising term 

end

function layer_U_hadamard1!(ms, triplets::Vector{Vector{Int64}})
    for trip in triplets
        U_hadamard1!(ms,trip[1],trip[2],trip[3])
    end
end

tripswap=CliffordOperator([QuantumClifford.P"_X_", QuantumClifford.P"__X", QuantumClifford.P"X__", QuantumClifford.P"_Z_", QuantumClifford.P"__Z", QuantumClifford.P"Z__"])

function U_tripswap!(ms::MixedDestabilizer,i::Integer,j::Integer,k::Integer)
    apply!(ms,tripswap,[i,j,k]) #kick on i
end
   

function layer_U_tripswap!(ms, triplets::Vector{Vector{Int64}})
    for trip in triplets
        U_tripswap!(ms,trip[1],trip[2],trip[3])
    end
end


Next we define our solvable initial state:

In [ ]:
function ghz(triplets::Vector{Vector{Int64}}, N::Integer)
    tripstab=PauliOperator{Array{UInt8, 0}, Vector{UInt64}}[]
    for trip in triplets
        push!(tripstab, pauli_ij(PauliZ,PauliZ,trip[1],trip[2],N))
        push!(tripstab, pauli_ij(PauliZ,PauliZ,trip[2],trip[3],N))
        push!(tripstab, pauli_ij(PauliX,PauliX,trip[1],trip[2],N)*pauli_i(PauliX,trip[3],N))
    end
    # singstab=S"-XX\n-ZZ"
    # s=reduce(⊗,repeat([singstab],N))
    return MixedDestabilizer(Stabilizer(tripstab))
end

The solvable regions in which we calculate the entanglement entropy correspond to geometric light cones grown from any point. Here is a function which calculates the entanglement entropy in such a region:

In [ ]:
function entanglement_lc(state::MixedDestabilizer, rad::Integer, z::Integer)
    entanglement_entropy(state,find_lc_zsite(z,rad+2,rad), Val(:rref))
end

In [ ]:
z=3
gens=10
trip_lists=graph_nodes_zsites(z,gens)

N=numnodes(z,gens)
state=ghz(trip_lists[1],N) #note where ghz's are

Nlayers=7
rad=6

ees_clifford=zeros(Int64,Nlayers+1)
ees_clifford[1]=entanglement_lc(state,rad,z)

for a in 1:Nlayers
    # apply layers of any type:

    layer_U_hadamard1!(state,trip_lists[a%2+1])
    # layer_U_kim!(state,trip_lists[a%2+1]) 
    # layer_U_tripswap!(state,trip_lists[a%2+1]) 

    ees_clifford[a+1]=entanglement_lc(state,rad,z)
end

ees_clifford

Let's check that this matches with the analytic formulae.
Note that we need to be a little careful with whether the radius is even or odd (corresponding to whether the initial EE is 0 or nonzero).

In [ ]:
s1(r,t)=3*2^r *(1-2.0^(-t)) 
s2(r,t)=3*2^r *(1-2.0^(-(t+1))) 

#if the radius is even
s_odd(r,t)= t%2==0 ? s1(r,t) : s2(r,t)
s_even(r,t) = t%2==0 ? s2(r,t) : s1(r,t)

#
s(r,t) = r%2==0 ? s_even(r,t) : s_odd(r,t)

In [ ]:
z=3
r=6

tvals=Int64[]
svals=Int64[]

#for t<r use the formula above
for t in 0:(r-1)
    push!(tvals,t)
    push!(svals, s(r,t))
end

#saturates from t=r
for t in r:r+3
    push!(tvals,t)
    push!(svals, s1(r,r))
end
svals